# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
df = pd.read_csv('event_datafile_new.csv')
df.to_csv('dataset')

In [5]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
keyspace_query = """CREATE KEYSPACE IF NOT EXISTS sparkify 
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

# Creating Keyspace
try:
    session.execute(keyspace_query)
except Exception as e:
    print(f"Failed to create keyspaceو Error : {e}")

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# QUERY 1

#### In this query we will need to filter by sessionId and itemInSession. and the CQL query looks like:

#### SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4

#### - We will name our table session_item
#### - Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.
#### - The columns of our table will be: artist, song, length, sessionId, itemInSession

In [9]:

query1 = """CREATE TABLE IF NOT EXISTS session_item (sessionId int, itemInSession int , artist text, song text, length float, \
PRIMARY KEY (sessionId, itemInSession))"""

session.execute(query1)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (sessionId, itemInSession,artist, song, length ) "
        query = query + "VALUES (%s, %s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0],line[9],float(line[5])) )


#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
select_query= "SELECT artist, song,length FROM session_item WHERE sessionId= 338 and itemInSession=4"
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


## Query 2

### The CQL our query looks like:

###### SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182

###### - We will name our table user_session
###### - Our primary key will consist of composite partition key userId, sessionId. Because if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes, which will have the performance issue when the volume of data is large.

###### - The columns of our table will be: userId,sessionId,itemInSession, artist, song, firstName, lastName

In [15]:
query2 = """CREATE TABLE IF NOT EXISTS user_session (userId int, sessionId int,itemInSession int,artist text, song text, firstName text, lastName text, PRIMARY KEY ( (userId, sessionId), itemInSession)) """

try: 
    session.execute(query2)
    print("Table Created")
except Exception as e:
    print(f"Table creation failed,Error : {e}")

Table Created


In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_session (userId,sessionId,itemInSession, artist, song, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))                  

In [30]:
select_query = "SELECT artist, song, firstName, lastName FROM  user_session where sessionId = 182 and userId = 10"
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3

#### The CQL query looks like:

##### SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'

###### - We will name our table song_history
###### - Our primary key will consist of partition key song_title, and clustering key user_id. This uniquely identifies our rows.
###### - The columns of our table will be: first_name, last_name, song_title, user_id

In [26]:
query3 = "CREATE TABLE IF NOT EXISTS song_history (song_title text, user_id int,first_name text, last_name text, \
PRIMARY KEY (song_title, user_id))"

session.execute(query3)
    

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (song_title, user_id,first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))  

In [28]:
query = "select first_name, last_name from song_history WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [29]:
delete_query1 = "DROP TABLE IF EXISTS session_item"
session.execute(delete_query1)

delete_query2 = "DROP TABLE IF EXISTS user_session"
session.execute(delete_query2)

delete_query3 = "DROP TABLE IF EXISTS song_history"
session.execute(delete_query3)

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()